<a href="https://colab.research.google.com/github/RanieryAV/Topicos-II-Primeiro-Trabalho/blob/main/Arvore-Geradora-Minima/AVL/AVL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import pickle
import tracemalloc
import timeit
from time import time_ns
import psutil
import os

In [ ]:
pip install pyavl3

In [ ]:
import pyavl3

###Dados

In [ ]:
arquivos1 = ['/content/dados/alue2087.stp','/content/dados/alue2105.stp','/content/dados/alue3146.stp','/content/dados/alue5067.stp','/content/dados/alue5345.stp','/content/dados/alue5623.stp','/content/dados/alue5901.stp','/content/dados/alue6179.stp','/content/dados/alue6457.stp','/content/dados/alue6735.stp','/content/dados/alue6951.stp','/content/dados/alue7065.stp','/content/dados/alue7066.stp','/content/dados/alue7080.stp', '/content/dados/alue7229.stp']

In [ ]:
arquivos2 = ['/content/dados/alut0787.stp','/content/dados/alut0805.stp','/content/dados/alut1181.stp','/content/dados/alut2010.stp','/content/dados/alut2288.stp','/content/dados/alut2566.stp','/content/dados/alut2610.stp','/content/dados/alut2625.stp','/content/dados/alut2764.stp']

In [ ]:
arquivos3 = ['/content/dados/dmxa0296.stp','/content/dados/dmxa0368.stp','/content/dados/dmxa0454.stp','/content/dados/dmxa0628.stp','/content/dados/dmxa0734.stp','/content/dados/dmxa0848.stp','/content/dados/dmxa0903.stp','/content/dados/dmxa1010.stp','/content/dados/dmxa1109.stp','/content/dados/dmxa1200.stp','/content/dados/dmxa1304.stp','/content/dados/dmxa1516.stp','/content/dados/dmxa1721.stp','/content/dados/dmxa1801.stp']

##Função do algoritmo de Prim

In [ ]:
def prim(graph, start_node):  # Define a função prim
    mst = []  # Inicializa a árvore geradora mínima como uma lista vazia
    visited = set([start_node])  # Inicializa o conjunto de nós visitados com o nó inicial
    avl_tree = pyavl3.AVLTree()  # Cria uma árvore AVL vazia

    # Cria uma lista de arestas do nó inicial e seus custos
    edges = [
        (cost, start_node, to)
        for to, cost in graph[start_node].items()
    ]

    # Insere todas as arestas na árvore AVL com o custo como chave
    for edge in edges:
        avl_tree[edge] = edge[0]

    # Continua enquanto houver arestas na árvore AVL
    while len(avl_tree) > 0:
        # Obtém e remove a aresta de menor custo da árvore AVL
        cost, frm, to = avl_tree._get_min(avl_tree.root).key
        avl_tree.pop((cost, frm, to))

        # Se o nó 'to' ainda não foi visitado
        if to not in visited:
            visited.add(to)  # Adiciona o nó 'to' ao conjunto de nós visitados
            mst.append((frm, to, cost))  # Adiciona a aresta à árvore geradora mínima

            # Para cada nó adjacente ao nó 'to'
            for to_next, cost2 in graph[to].items():
                # Se o nó adjacente ainda não foi visitado
                if to_next not in visited:
                    # Adiciona a aresta à árvore AVL
                    avl_tree[(cost2, to, to_next)] = cost2

    return mst # Retorna a árvore geradora mínima

##Função para ler o arquivo .stp e traduzir para uma estrutura de dicionário representando um grafo

In [ ]:
def read_stp(filename):
    graph = {} #graph será um dicionário cujas chaves são os números dos nós e os valores são outros dicionários que por sua vez têm como chaves os nós ligados aos nós das chaves mais externas e seus valores sendo os pesos das arestas que os interligam
    with open(filename, 'r') as file:
        for line in file:
            if line.startswith('E '): #percorre as linhas do arquivo manipulando apenas as que iniciam com "E " indicando ser uma aresta do grafo
                data = line.strip().split()
                node1 = int(data[1])
                node2 = int(data[2])
                cost = float(data[3])
                if node1 not in graph:
                    graph[node1] = {}
                if node2 not in graph:
                    graph[node2] = {}
                graph[node1][node2] = cost
                graph[node2][node1] = cost
    return graph

##Função principal que usa as anteriores para ler o arquivo de entrada, modelar o grafo e retornar a MST

In [ ]:
def main():
    filename = '/content/dmxa0296.stp'
    graph = read_stp(filename)
    mst, _ = prim(graph, 1)
    print(mst) #A Mst é retornada como uma lista de tuplas, cada tupla contém a aresta que é representada pelos seus vértices interligados e respectivo peso

In [ ]:
if __name__ == "__main__":
    main()

In [ ]:
len(mst)

##Funções para automatizar o treino e plotagem de gráficos

In [ ]:
def avl_dataAuto(arquivos):
  memoryList = [] #Lista das memórias usadas nas execuções para cada entrada
  timeList = [] #Lista dos tempos decorridos nas execuções para cada entrada
  tracemalloc.start() #inicia a medição de memória
  for arquivo in (arquivos): #percorre todos os arquivos passados como entrada
    graph = read_stp(arquivo) #traduz o grafo presente no arquivo

    start_time = time_ns() #inicia a contagem do tempo da execução atual
    mst = prim(graph, 1) #chama a função de prim
    totalMemory, _ = tracemalloc.get_traced_memory() #obtém o valor da memória total usada na execução da função
    end_time = time_ns() #finaliza a contagem do tempo da execução atual

    tracemalloc.reset_peak() #reseta a medição da memória

    elapsed_time = end_time - start_time #cálculo do tempo total de execução

    memoryList.append(totalMemory) #atualiza a lista de memórias
    timeList.append(elapsed_time) #atualiza a lista de tempos

  return memoryList, timeList

###Função para determinar um tempo mínimo de execução dos testes

In [ ]:
def loop(arquivos):
  avl_data = {"memoria":{}, "tempo":{}} #dicionário para guardar os valores médios da medição de tempo e memória

  iteracoes = 0 #para contar o número de iterações possibilitando o cálculo de tempo e memória médios
  tempo_total = 0 #variável de controle do laço para detectar o cumprimento dos 5 segundos mínimos de execução
  tempo_parada = 5*10**9 #constante que representa os 5 segundos (em ns)
  memoria_soma = [0]*len(arquivos) #lista para guardar a soma das memórias gastas em cada iteração
  memorias = [] #Para guardar as médias das memórias
  tempos = [] #Para guardar as médias dos tempos
  tempo_soma = [0]*len(arquivos) #lista para guardar a soma dos tempos decorridos em cada iteração
  tempo_inicial = time_ns() #início da contagem do tempo relativo aos 5 segundos mínimos de execução
  while tempo_total < tempo_parada: #laço para garantir o término das execuções com no mínimo 5 segundos decorridos
    memoria, tempo = avl_dataAuto(arquivos) #pega as listas de memória e tempo da iteração corrente
    memoria_soma = [mem1 + mem2 for mem1, mem2 in zip(memoria_soma, memoria)] #Atualiza a lista das somas das memórias
    tempo_soma = [tem1 + tem2 for tem1, tem2 in zip(tempo_soma, tempo)] #Atualiza a lista das somas dos tempos
    tempo_final = time_ns() #ponto de verificação do tempo decorrido do laço enquanto
    tempo_total = tempo_final - tempo_inicial #cálculo do tempo total decorrido
    iteracoes += 1 #incremento do número de iterações

  memorias = [item/iteracoes for item in memoria_soma] #calcula a média das memórias
  tempos = [item/iteracoes for item in tempo_soma] #calcula a média dos tempos

  for arquivo, memoria, tempo in zip(arquivos, memorias, tempos): #Laço para formação do dicionário contendo as informações de memória e tempo médios da base testada
    avl_data['memoria'][arquivo] = memoria
    avl_data['tempo'][arquivo] = tempo

  return avl_data, iteracoes

dic_avl, iteracoes = loop(arquivos2) #execução da função de teste que garante os mínimos 5 segundos de teste

with open(f"avl_alut({iteracoes}).pkl", "wb") as f:
    # Usa a função dump para salvar o dicionário no arquivo
    pickle.dump(dic_avl, f)


In [ ]:
dic_avl

{'memoria': {'/content/dados/alut0787.stp': 550698.0,
  '/content/dados/alut0805.stp': 544434.0,
  '/content/dados/alut1181.stp': 1609893.0,
  '/content/dados/alut2010.stp': 3193582.0,
  '/content/dados/alut2288.stp': 4555616.0,
  '/content/dados/alut2566.stp': 2596207.0,
  '/content/dados/alut2610.stp': 16834361.0,
  '/content/dados/alut2625.stp': 18137433.0,
  '/content/dados/alut2764.stp': 382268.0},
 'tempo': {'/content/dados/alut0787.stp': 219474640.0,
  '/content/dados/alut0805.stp': 149011582.0,
  '/content/dados/alut1181.stp': 600609300.0,
  '/content/dados/alut2010.stp': 1267798381.0,
  '/content/dados/alut2288.stp': 1897966637.0,
  '/content/dados/alut2566.stp': 1582749239.0,
  '/content/dados/alut2610.stp': 7860923753.0,
  '/content/dados/alut2625.stp': 9342545866.0,
  '/content/dados/alut2764.stp': 88876498.0}}

In [ ]:
nome_arquivo = f"avl_alue({iteracoes}).pkl"
with open(nome_arquivo, 'rb') as arquivo:
    avl_time = pickle.load(arquivo)

In [ ]:
import matplotlib.pyplot as plt

x = range(1, len(avl_time) + 1)
y = avl_time.values()

plt.plot(x, y)
plt.title('Tempo gasto na execução de Prim usando AVL')
plt.xlabel('Dados')
plt.ylabel('Tempo(ns)')
plt.xticks(x)
plt.show()

In [ ]:
x = range(1, len(avl_space) + 1)
y = avl_space.values()

plt.plot(x, y)
plt.title('Memória gasta na execução de Prim usando AVL')
plt.xlabel('Dados')
plt.ylabel('Memória(B)')
plt.xticks(x)
plt.show()


##Exemplo de como é lido e interpretado o grafo de um arquivo de entrada

In [ ]:
filename = '/content/dmxa0296.stp'
graph = read_stp(filename)

In [ ]:
for to, cost in graph[1].items():
  print(f"to: {to}; cost: {cost}")

to: 2; cost: 5.0
to: 14; cost: 13.0


In [ ]:
#avl_tree.update(graph)
avl_tree = pyavl3.AVLTree()

# Adiciona alguns nós à árvore AVL
avl_tree[1] = 2
avl_tree[2] = 4
avl_tree[3] = 5
avl_tree[4] = 3

In [ ]:
print(avl_tree)

<AVL {3: 5, 2: 4, 4: 3}>


In [ ]:
avl_tree.pop(1) #Remove a chave e retorna o valor

2

In [ ]:
avl_tree._get_min(avl_tree.root).value #valor na menor chave, mas trocar value por key retorna a mínima chave

2

In [ ]:
#Encontrar o mínimo valor na arvore
def find_min(tree):
    node = tree.root
    while node.left is not None:
        node = node.left
    return node.key, node.value

<Node key=54, value={40: 13.0, 53: 5.0, 55: 5.0, 67: 13.0}, height=6>

In [ ]:
avl_tree.copy

{1: 5.0, 15: 13.0}

In [ ]:
tempo_total = 0 #variável de controle do laço para detectar o cumprimento dos 5 segundos mínimos de execução
tempo_parada = 5*10**9 #constante que representa os 5 segundos (em ns)
tempo_inicial = time_ns() #início da contagem do tempo relativo aos 5 segundos mínimos de execução
while tempo_total < tempo_parada: #laço para garantir o término das execuções com no mínimo 5 segundos decorridos
  for i in range(1000):
    pass
  tempo_final = time_ns() #ponto de verificação do tempo decorrido do laço enquanto
  tempo_total = tempo_final - tempo_inicial #cálculo do tempo total decorrido